# Carico il dataset

In [1]:
import small_DS_functions as sd
from datetime import *
import swifter
sd.steam.info()
sd.steam.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   app_id                          20000 non-null  int64         
 1   app_name                        20000 non-null  object        
 2   review_id                       20000 non-null  int64         
 3   language                        20000 non-null  object        
 4   review                          19934 non-null  object        
 5   timestamp_created               20000 non-null  datetime64[ns]
 6   timestamp_updated               20000 non-null  datetime64[ns]
 7   recommended                     20000 non-null  bool          
 8   votes_helpful                   20000 non-null  int64         
 9   votes_funny                     20000 non-null  int64         
 10  weighted_vote_score             20000 non-null  float64       
 11  co

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,2021-01-23 06:00:29,2021-01-23 06:00:29,True,0,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,2021-01-22 19:23:03


# Testing

In [ ]:
sd.steam[(sd.steam["review"].isnull())]

In [ ]:
sd.steam.describe()
sd.steam["app_name"].describe()
sd.steam["author.num_games_owned"].describe()

In [ ]:
corr = sd.steam.corr()
corr

In [ ]:
fig = sd.plt.figure(figsize=(15,15))
sd.plt.matshow(corr, cmap="RdBu", fignum=fig.number)
sd.plt.xticks(range(len(corr.columns)), corr.columns, rotation="vertical");
sd.plt.yticks(range(len(corr.columns)), corr.columns);

In [ ]:
sd.steam.info()
sd.steam["app_name"].drop_duplicates()

# RQ3

*Now it's important to understand the preferred time to do reviews.*
   - What is the most common time that authors review an application? For example, authors usually write a review at 17:44.


In [ ]:
sd.plt.figure(1, figsize=(60,20))
barM, dicM = sd.numbersOfReviewForTime(sd.datetime_range(0,23,1))
print(sd.Counter(dicM).most_common(3))
sd.plt.figure(2, figsize=(60,20))
barH, dicH = sd.numbersOfReviewForTime(sd.datetime_range(0,23,30))
print(sd.Counter(dicH).most_common(3))

   - Create a function that receives as a parameter a list of time intervals and returns the plot the number of reviews for each of the intervals.


In [ ]:
sd.numbersOfReviewForTime([["8:00", "10:00"],["13:00", "18:00"]])

    - Use the function that you created in the previous literal to plot the number of reviews between the following time intervals: 
    
| Initial time | Final time |
| ----------- | ----------- |
|06:00:00 | 10:59:59|
|11:00:00 | 13:59:59|
|14:00:00 | 16:59:59|
|17:00:00 | 19:59:59|
|20:00:00 | 23:59:59|
|00:00:00 | 02:59:59|
|03:00:00 | 05:59:59|

In [ ]:
sd.plt.figure(figsize=(15,15))
sd.numbersOfReviewForTime([["06:00:00", "10:59:59"],["11:00:00", "13:59:59"], ["14:00:00",  "16:59:59"], ["17:00:00", "19:59:59"], ["20:00:00", "23:59:59"], ["00:00:00", "02:59:59"],["03:00:00", "05:59:59"]]);

# RQ4

In [ ]:
def reviewsForLanguages(dataset, languages):
    dataout = dataset[(dataset.language == languages[0])]
    
    for x in languages[1:]:
        
        dataout = sd.pd.concat([dataset[(dataset.language == x)], dataout], ignore_index = True)
        
    return dataout
    

In [ ]:
reviewsForLanguages(sd.steam, ["english", "italian"])

# RQ5

In [ ]:
sd.steam[["weighted_vote_score", "votes_helpful", "votes_funny"]]

# RQ6 test
   - What is the average time (days and minutes) a user lets pass before he updates a review?


In [6]:
t = sd.steam[["timestamp_created", "timestamp_updated"]].swifter.apply(lambda row:row["timestamp_updated"] - row["timestamp_created"], axis=1)

In [4]:
t.describe()

count                        20000
mean        0 days 04:24:01.345350
std      1 days 15:47:35.540102217
min                0 days 00:00:00
25%                0 days 00:00:00
50%                0 days 00:00:00
75%                0 days 00:00:00
max               37 days 12:05:41
dtype: object

### NOTA

  - C'è un solo valore negativo: created < update ????

In [ ]:
t.sort_values()

   - Plot the top 3 authors that usually update their reviews.
   
Top 3 utenti con più recensioni aggiornate (Questa lista non prende in considerazione il numero totale di recensioni per utente: un utente con più recensioni si troverà probabilmente più in alto in classifica)

In [ ]:
author_steamId_AND_timedelta = df.steam[["author.steamid"]]
author_steamId_AND_timedelta["Numero Modifiche"] = t

In [ ]:
authorMOD = author_steamId_AND_timedelta.loc[author_steamId_AND_timedelta["Numero Modifiche"] != "0", ["author.steamid", "Numero Modifiche"]]
authorMOD = authorMOD.groupby(["author.steamid"]).count()
authorMOD

In [ ]:
authorMOD_sortedHead = authorMOD.sort_values(["Numero Modifiche"], ascending=False).head(3)
authorMOD_sortedHead
df.plt.bar(list(map(str,authorMOD_sortedHead.index)), authorMOD_sortedHead["Numero Modifiche"]);

In [ ]:
authorMOD_sortedHead

Top 3 utenti con più recensioni aggiornate (Questa lista prende in considerazione il numero totale di recensioni e il numero di recensioni aggiornate)

In [ ]:
authorNOMOD = author_steamId_AND_timedelta.loc[author_steamId_AND_timedelta["Numero Modifiche"] == "0", ["author.steamid", "Numero Modifiche"]]
authorNOMOD = authorNOMOD.rename({"Numero Modifiche": "Numero non modifiche"}, axis=1)
authorNOMOD = authorNOMOD.groupby(["author.steamid"]).count()
authorNOMOD

In [ ]:
AuthorTOT = author_steamId_AND_timedelta.groupby(["author.steamid"]).count()
AuthorTOT = AuthorTOT.rename({"Numero Modifiche": "TOT"}, axis=1)
AuthorTOT["Numero modifiche"] = authorMOD
AuthorTOT["Numero non modifiche"] = authorNOMOD
AuthorTOT.head(10)

In [ ]:
perc = AuthorTOT.apply(lambda row: row["Numero modifiche"]*100/row["TOT"],axis=1)
AuthorTOT["Percentuale di modifica"] = perc
AuthorTOT

In [ ]:
x =AuthorTOT.sort_values(["Percentuale di modifica", "TOT"], ascending=False).head(3)
x